In [1]:
# Stock price source - yahoo finance 
#Import dependencies
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Read stock price csv files 
zoom = pd.read_csv("Resources/zoom.csv")
slack = pd.read_csv("Resources/slack.csv")
shopify = pd.read_csv("Resources/shopify.csv")
cisco = pd.read_csv("Resources/cisco.csv")

#adjust date format in shopify dataframe

shopify["Date"]=pd.to_datetime(shopify["Date"])
shopify["Date"]=shopify["Date"].dt.strftime("%Y-%m-%d")

In [3]:
#Remove other columns in the df, only keep data and closing price

zoom=zoom[["Date","Close"]]
slack=slack[["Date","Close"]]
cisco=cisco[["Date","Close"]]
shopify=shopify[["Date","Close"]]

In [4]:
#Merge different stock price dataframe into a combined dataframe

combined_df = pd.merge(zoom, slack, how='outer', on="Date", suffixes=["_Zoom", "_Slack"])
combined_df = pd.merge(combined_df, cisco, how='outer', on="Date", suffixes=["_Slack","_Cisco"])
combined_df = pd.merge(combined_df, shopify, how='outer', on="Date",  suffixes=["_Cisco", "_Shopify"])

#change the names for columns

combined_df = combined_df.rename(columns = {"Close_Zoom":"Zoom","Close_Slack":"Slack","Close_Cisco":"Cisco","Close_Shopify":"Shopify"})
combined_df[["Zoom","Slack","Cisco","Shopify"]]=combined_df[["Zoom","Slack","Cisco","Shopify"]].round(2)

#Display combined dataframe
combined_df.head()

,Date,Zoom,Slack,Cisco,Shopify
0,2019-10-31,69.89,22.00,47.51,313.57
1,2019-11-01,70.39,21.94,47.03,316.76
2,2019-11-04,70.12,21.94,47.48,307.91
3,2019-11-05,66.89,20.75,47.76,295.50
4,2019-11-06,66.83,20.14,48.27,295.02
